# Working with Azure Machine Learning Studio engine

This notebook shows how to monitor the model deployed on Azure Machine Learning Studio using Watson OpenScale python sdk.

Contents

1. [Setup](#setup)
2. [Binding machine learning engine](#binding)
3. [Subscriptions](#subsciption)
4. [Performance monitor, scoring and payload logging](#scoring)
5. [Quality monitor and feedback logging](#feedback)
6. [Fairness, Drift monitor and explainability](#monitors)

<a id="setup"></a>
# 1. Setup

### Sample model creation using [Azure Machine Learning Studio](https://studio.azureml.net)

- Copy [credit risk experiment](https://gallery.cortanaintelligence.com/Experiment/German-credit-risk-created-on-1-9-2019) from Azure ML Studio Gallery 
- Run experiment to train a model
- Create (deploy) web service (new)

**NOTE:** Classic web services are not supported.

### Installation and authentication

In [3]:
!pip install --upgrade ibm-cloud-sdk-core --no-cache | tail -n 1
!pip install --upgrade --upgrade ibm-watson-openscale --no-cache | tail -n 1
!pip install --upgrade pyspark==2.4 --no-cache | tail -n 1
!pip install --upgrade lime --no-cache | tail -n 1

**Action:** Restart the kernel (Kernel->Restart)

#### ACTION: Get Watson OpenScale `apikey`

How to install IBM Cloud (bluemix) console: [instruction](https://console.bluemix.net/docs/cli/reference/ibmcloud/download_cli.html#install_use)

How to get api key using bluemix console:
```
bx login --sso
bx iam api-key-create 'my_key'
```

In [1]:
CLOUD_API_KEY = 'wqMwzhQ7GRNBlH5iS3OVGlf3Ym4OD4EPJjaA6Eg8mQgg'

In [5]:
DB_CREDENTIALS=None
#DB_CREDENTIALS= {"hostname":"","username":"","password":"","database":"","port":"","ssl":True,"sslmode":"","certificate_base64":""}

In [6]:
SCHEMA_NAME = 'azure_dm'

In [12]:
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator,BearerTokenAuthenticator

from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *


authenticator = IAMAuthenticator(apikey=CLOUD_API_KEY)
#authenticator = BearerTokenAuthenticator(bearer_token=IAM_TOKEN) ## uncomment if using IAM token
wos_client = APIClient(authenticator=authenticator)
wos_client.version

'3.0.2'

#### Download and preview training data set

In [2]:
!rm -rf credit_risk_training.csv
!wget "https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/assets/data/credit_risk/credit_risk_training.csv"

--2020-11-03 14:13:24--  https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/assets/data/credit_risk/credit_risk_training.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.124.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.124.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 694222 (678K) [text/plain]
Saving to: ‘credit_risk_training.csv’

credit_risk_trainin 100%[===================>] 677.95K  3.44MB/s    in 0.2s    

2020-11-03 14:13:25 (3.44 MB/s) - ‘credit_risk_training.csv’ saved [694222/694222]



In [3]:
import pandas as pd

training_data_pd = pd.read_csv(
        "credit_risk_training.csv",
        dtype={'LoanDuration': int, 'LoanAmount': int, 'InstallmentPercent': int, 'CurrentResidenceDuration': int, 'Age': int, 'ExistingCreditsCount': int, 'Dependents': int})

In [4]:
training_data_pd.head()

,CheckingStatus,LoanDuration,CreditHistory,LoanPurpose,LoanAmount,ExistingSavings,EmploymentDuration,InstallmentPercent,Sex,OthersOnLoan,...,OwnsProperty,Age,InstallmentPlans,Housing,ExistingCreditsCount,Job,Dependents,Telephone,ForeignWorker,Risk
0,0_to_200,31,credits_paid_to_date,other,1889,100_to_500,less_1,3,female,none,...,savings_insurance,32,none,own,1,skilled,1,none,yes,No Risk
1,less_0,18,credits_paid_to_date,car_new,462,less_100,1_to_4,2,female,none,...,savings_insurance,37,stores,own,2,skilled,1,none,yes,No Risk
2,less_0,15,prior_payments_delayed,furniture,250,less_100,1_to_4,2,male,none,...,real_estate,28,none,own,2,skilled,1,yes,no,No Risk
3,0_to_200,28,credits_paid_to_date,retraining,3693,less_100,greater_7,3,male,none,...,savings_insurance,32,none,own,1,skilled,1,none,yes,No Risk
4,no_checking,28,prior_payments_delayed,education,6235,500_to_1000,greater_7,3,male,none,...,unknown,57,none,own,2,skilled,1,none,yes,Risk


## Save training data to IBM Cloud Object Storage

In next cells, you will need to paste some credentials to Cloud Object Storage. If you haven't worked with COS yet please visit getting started with COS tutorial. You can find COS_API_KEY_ID and COS_RESOURCE_CRN variables in Service Credentials in menu of your COS instance. Used COS Service Credentials must be created with Role parameter set as Writer. Later training data file will be loaded to the bucket of your instance and used as training refecence in subsription. COS_ENDPOINT variable can be found in Endpoint field of the menu.

In [5]:
IAM_URL="https://iam.ng.bluemix.net/oidc/token"

In [6]:
COS_API_KEY_ID = "917Z-0MQzVpgdqkHClXbfDchrXZ_bl7kbCxZSkynzsLP"
COS_RESOURCE_CRN = "crn:v1:bluemix:public:cloud-object-storage:global:a/e0b56432b1f1bd804706dc29b8a89ca1:57b5eb6e-7b5d-4b90-a8e8-3736129c9010::" # eg "crn:v1:bluemix:public:cloud-object-storage:global:a/3bf0d9003abfb5d29761c3e97696b71c:d6f04d83-6c4f-4a62-a165-696756d63903::"
COS_ENDPOINT = "https://s3.us.cloud-object-storage.appdomain.cloud" # Current list avaiable at https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints

In [7]:
BUCKET_NAME = "wmlv4-donotdelete-pr-e8rgr9bhcjtlae" #example: "credit-risk-training-data"
training_data_file_name="credit_risk_training.csv"

In [8]:
import ibm_boto3
from ibm_botocore.client import Config, ClientError

cos_client = ibm_boto3.resource("s3",
    ibm_api_key_id=COS_API_KEY_ID,
    ibm_service_instance_id=COS_RESOURCE_CRN,
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version="oauth"),
    endpoint_url=COS_ENDPOINT
)

In [9]:
with open(training_data_file_name, "rb") as file_data:
    cos_client.Object(BUCKET_NAME, training_data_file_name).upload_fileobj(
        Fileobj=file_data
    )

### DataMart setup

In [13]:
wos_client.data_marts.show()

WOS Data Mart,Data Mart created by WOS tutorial notebook,True,active,2020-07-22 22:17:14.701000+00:00,5a0b9076-fcf6-49e8-a824-9e3a6b4c2a56


In [14]:
data_marts = wos_client.data_marts.list().result.data_marts
if len(data_marts) == 0:
    if DB_CREDENTIALS is not None:
        if SCHEMA_NAME is None: 
            print("Please specify the SCHEMA_NAME and rerun the cell")

        print('Setting up external datamart')
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook",
                database_configuration=DatabaseConfigurationRequest(
                  database_type=DatabaseType.POSTGRESQL,
                    credentials=PrimaryStorageCredentialsLong(
                        hostname=DB_CREDENTIALS['hostname'],
                        username=DB_CREDENTIALS['username'],
                        password=DB_CREDENTIALS['password'],
                        db=DB_CREDENTIALS['database'],
                        port=DB_CREDENTIALS['port'],
                        ssl=True,
                        sslmode=DB_CREDENTIALS['sslmode'],
                        certificate_base64=DB_CREDENTIALS['certificate_base64']
                    ),
                    location=LocationSchemaName(
                        schema_name= SCHEMA_NAME
                    )
                )
             ).result
    else:
        print('Setting up internal datamart')
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook", 
                internal_database = True).result
        
    data_mart_id = added_data_mart_result.metadata.id
    
else:
    data_mart_id=data_marts[0].metadata.id
    print('Using existing datamart {}'.format(data_mart_id))

Using existing datamart 5a0b9076-fcf6-49e8-a824-9e3a6b4c2a56


<a id="binding"></a>
# 2. Bind machine learning engines

### Bind  `Azure` machine learning studio

Provide credentials using following fields:
- `client_id`
- `client_secret`
- `subscription_id`
- `tenant`

In [15]:
AZURE_ENGINE_CREDENTIALS = {
    "client_id": "c943757b-86e7-417c-b3e4-45e2738fe59c",
    "client_secret": "5c975694-9d72-4c42-9bef-5255bdb8475c",
    "subscription_id": "744bca72-2299-451c-b682-ed6fb75fb671",
    "tenant": "fcf67057-50c9-4ad4-98f3-ffca64add9e9"
}

In [16]:
SERVICE_PROVIDER_NAME = "Azure Machine Learning"
SERVICE_PROVIDER_DESCRIPTION = "Added by Azure tutorial WOS notebook."

In [18]:
service_providers = wos_client.service_providers.list().result.service_providers
for service_provider in service_providers:
    service_instance_name = service_provider.entity.name
    if service_instance_name == SERVICE_PROVIDER_NAME:
        service_provider_id = service_provider.metadata.id
        wos_client.service_providers.delete(service_provider_id)
        print("Deleted existing service_provider for WML instance: {}".format(service_provider_id))

In [19]:
added_service_provider_result = wos_client.service_providers.add(
        name=SERVICE_PROVIDER_NAME,
        description=SERVICE_PROVIDER_DESCRIPTION,
        service_type=ServiceTypes.AZURE_MACHINE_LEARNING,
        credentials=AzureCredentials(
            subscription_id= AZURE_ENGINE_CREDENTIALS['subscription_id'], 
            client_id = AZURE_ENGINE_CREDENTIALS['client_id'], 
            client_secret= AZURE_ENGINE_CREDENTIALS['client_secret'],
            tenant = AZURE_ENGINE_CREDENTIALS['tenant']
        ),
        background_mode=False
    ).result
service_provider_id = added_service_provider_result.metadata.id




 Waiting for end of adding service provider f8ef24c2-44e5-4a3d-bebb-7fde59a1cf51 




active

-----------------------------------------------
 Successfully finished adding service provider 
-----------------------------------------------




In [20]:
wos_client.service_providers.show()

None,active,Azure Machine Learning,azure_machine_learning,2020-11-03 19:14:47.650000+00:00,f8ef24c2-44e5-4a3d-bebb-7fde59a1cf51
128b6597-d502-48b9-ba30-8e5a801150ea,active,Watson Machine Learning V2,watson_machine_learning,2020-11-02 23:59:49.307000+00:00,6763d6c4-a40e-4b1b-a58f-7c18b56b37a5
128b6597-d502-48b9-ba30-8e5a801150ea,active,Watson Machine Learning V2 test,watson_machine_learning,2020-11-02 22:56:33.856000+00:00,33814b15-fd8a-47ff-9302-a78b2062d1ac
None,active,Watson Machine Learning V2 SPSS,spss_collaboration_and_deployment_services,2020-10-28 18:41:21.119000+00:00,858febc8-a777-4d60-a7c5-27cf380d4158
128b6597-d502-48b9-ba30-8e5a801150ea,active,Watson Machine Learning prod openpage,watson_machine_learning,2020-10-26 22:33:14.936000+00:00,c393b4e1-502a-4ead-a636-06bf03bd3425
128b6597-d502-48b9-ba30-8e5a801150ea,active,Watson Machine Learning pre-prod openpage,watson_machine_learning,2020-10-26 22:11:50.572000+00:00,fa7f0524-619e-4f7a-9bff-281853a81f2c
None,active,My Custom Engine,custom_machine_learning,2020-10-21 20:00:44.376000+00:00,5d4cede3-0d09-42da-903b-7283d553c882
None,active,My Custom Engine,custom_machine_learning,2020-10-20 22:00:15.137000+00:00,29edba0e-dc0e-486f-ba97-8a7e91cbf4d8


In [21]:
asset_deployment_details = wos_client.service_providers.list_assets(data_mart_id=data_mart_id, service_provider_id=service_provider_id).result
asset_deployment_details

{'resources': [{'metadata': {'guid': 'f33b3eddae0d51f790fe883f221bdaf2',
    'url': 'https://ussouthcentral.services.azureml.net:443/subscriptions/744bca722299451cb682ed6fb75fb671/services/acd8abca89e54be0a61a1da62a87224a/execute?api-version=2.0&format=swagger',
    'created_at': '2019-05-06T11:33:11.2195519Z',
    'modified_at': '2019-05-06T11:33:30.3979827Z'},
   'entity': {'name': 'adult.census.binary',
    'deployment_rn': '/subscriptions/744bca72-2299-451c-b682-ed6fb75fb671/resourceGroups/bias_explainability/providers/Microsoft.MachineLearning/webServices/adult.census.binary',
    'type': 'online',
    'description': '',
    'scoring_endpoint': {'url': 'https://ussouthcentral.services.azureml.net:443/subscriptions/744bca722299451cb682ed6fb75fb671/services/acd8abca89e54be0a61a1da62a87224a/execute?api-version=2.0&format=swagger',
     'credentials': {'token': '3glK6bPnKySmatyPOeh9gPvM4MaB8J8oOH/1Ea17Qm4A/KMLsUQ4angBmzH6GUGTLfN9dQMY/wUVNJ7UpgJYzA=='},
     'request_headers': {'Conten

In [22]:
## select your model deployment GUID and get model asset details
deployment_id='6145fc17a53dc0223d90aaf45b7cd2b7'#'9764f091ac3099d6c5182764c7b97f90'
for model_asset_details in asset_deployment_details['resources']:
    if model_asset_details['metadata']['guid']==deployment_id:
        break

### sample scoring

In [23]:
import requests
import time
import json

scoring_url = model_asset_details['entity']['scoring_endpoint']['url']

data = {
            "Inputs": {
                "input1":
                    [{'CheckingStatus': 'no_checking',
                      'LoanDuration': 19,
                      'CreditHistory': 'credits_paid_to_date',
                      'LoanPurpose': 'business',
                      'LoanAmount': 1867,
                      'ExistingSavings': '100_to_500',
                      'EmploymentDuration': '1_to_4',
                      'InstallmentPercent': 4,
                      'Sex': 'male',
                      'OthersOnLoan': 'none',
                      'CurrentResidenceDuration': 3,
                      'OwnsProperty': 'car_other',
                      'Age': 42,
                      'InstallmentPlans': 'stores',
                      'Housing': 'own',
                      'ExistingCreditsCount': 2,
                      'Job': 'unskilled',
                      'Dependents': 1,
                      'Telephone': 'none',
                      'ForeignWorker': 'yes'}]
            },
            "GlobalParameters": {
            }
        }

body = str.encode(json.dumps(data))

#token = subscription_details['entity']['deployment']['scoring_endpoint']['credentials']['secret_id']
token = model_asset_details['entity']['scoring_endpoint']['credentials']['token']
headers = model_asset_details['entity']['scoring_endpoint']['request_headers']
headers['Authorization'] = ('Bearer ' + token)

start_time = time.time()
response = requests.post(url=scoring_url, data=body, headers=headers)
response_time = int(time.time() - start_time)*1000
result = response.json()

print(json.dumps(result, indent=2))

{
  "Results": {
    "output1": [
      {
        "CheckingStatus": "no_checking",
        "LoanDuration": "19",
        "CreditHistory": "credits_paid_to_date",
        "LoanPurpose": "business",
        "LoanAmount": "1867",
        "ExistingSavings": "100_to_500",
        "EmploymentDuration": "1_to_4",
        "InstallmentPercent": "4",
        "Sex": "male",
        "OthersOnLoan": "none",
        "CurrentResidenceDuration": "3",
        "OwnsProperty": "car_other",
        "Age": "42",
        "InstallmentPlans": "stores",
        "Housing": "own",
        "ExistingCreditsCount": "2",
        "Job": "unskilled",
        "Dependents": "1",
        "Telephone": "none",
        "ForeignWorker": "yes",
        "Scored Labels": "No Risk",
        "Scored Probabilities": "0.000162622847710736"
      }
    ]
  }
}


<a id="subsciption"></a>
# 3. Subscriptions

### Add subscriptions

List available deployments.

**Note:** Depending on number of assets it may take some time.

In [24]:
wos_client.subscriptions.show()

ac19f6fd-ae79-46a9-9fd2-7bede4b8c313,house_price_xgbregression,5a0b9076-fcf6-49e8-a824-9e3a6b4c2a56,a9d12e3e-d251-43ab-bda6-500160babea7,house_price_xgbregression_deployment,6763d6c4-a40e-4b1b-a58f-7c18b56b37a5,active,2020-11-03 01:06:22.009000+00:00,a337d621-5de4-4c63-8d83-fd02d6db14af
d551ef44-2d40-4e5c-b5ad-4abf1b67a0ec,gosales_multiclass,5a0b9076-fcf6-49e8-a824-9e3a6b4c2a56,3ee743dc-20e9-4ff7-b46f-10939fff9ee0,house_price_xgbregression_deployment,33814b15-fd8a-47ff-9302-a78b2062d1ac,active,2020-11-02 22:56:49.711000+00:00,887596d0-87f9-4eaf-9ee9-30b87d5d03ba
9099ad7e60c9698597199983c976c65a,None,5a0b9076-fcf6-49e8-a824-9e3a6b4c2a56,91a58dcc30f1cf7ee72f06ac6f8f3c67,credit-risk-rsalehin,5d4cede3-0d09-42da-903b-7283d553c882,active,2020-10-21 20:54:17.435000+00:00,72a076d6-fb87-4f52-96b8-891cd405c6e1


In [25]:
feature_columns = ['CheckingStatus', 'LoanDuration', 'CreditHistory', 'LoanPurpose', 'LoanAmount', 'ExistingSavings', 'EmploymentDuration', 'InstallmentPercent', 'Sex', 'OthersOnLoan', 'CurrentResidenceDuration', 'OwnsProperty', 'Age', 'InstallmentPlans', 'Housing', 'ExistingCreditsCount', 'Job', 'Dependents', 'Telephone', 'ForeignWorker']
categorical_columns = ['CheckingStatus', 'CreditHistory', 'LoanPurpose', 'ExistingSavings', 'EmploymentDuration', 'Sex', 'OthersOnLoan', 'OwnsProperty', 'InstallmentPlans', 'Housing', 'Job', 'Telephone', 'ForeignWorker']

In [26]:
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import ScoringEndpointCredentialsAzureScoringEndpointCredentials,ScoringEndpointRequest
def get_output_schema_fields_for_azure(azure_ouput_schema, target_field, target_role = "decoded-target", field_type = str, is_nullable = True):
        fields = []
        found = False
        for field in azure_ouput_schema['fields']:
            if field['name']==target_field:
                metadata = {}
                if "metadata" in field:
                    metadata = field["metadata"]
                metadata["modeling_role"]=target_role    
                field["metadata"] = metadata
                found = True
    
            spark_field = SparkStructFieldObject(
                name=field['name'],
                type=field['type'],
                nullable=field['nullable'],
            )
            if "metadata" in field:
                spark_field.metadata = field["metadata"]
                
            fields.append(spark_field)   
        
        if found is False:
            metadata = {}
            metadata["modeling_role"] = target_role
            fields.append(SparkStructFieldObject(
                name=target_field,
                type=field_type,
                nullable=nullable,
                metadata = metadata
            )) 
        return fields

In [27]:
azure_asset = Asset(
            asset_id=model_asset_details["entity"]["asset"]["asset_id"],
            name=model_asset_details["entity"]["asset"]["name"],
            url=model_asset_details["entity"]["asset"]["url"],
            asset_type=AssetTypes.MODEL,
            input_data_type=InputDataType.STRUCTURED,
            problem_type=ProblemType.BINARY_CLASSIFICATION,
        )

In [28]:
deployment_scoring_endpoint = model_asset_details['entity']['scoring_endpoint']
scoring_endpoint = ScoringEndpointRequest(url = model_asset_details['entity']['scoring_endpoint']['url']
                                          ,request_headers = model_asset_details['entity']['scoring_endpoint']['request_headers'],
                                          credentials= ScoringEndpointCredentialsAzureScoringEndpointCredentials(model_asset_details['entity']['scoring_endpoint']['credentials']['token']) )

deployment = AssetDeploymentRequest(
    deployment_id=model_asset_details['metadata']['guid'],
    url=model_asset_details['metadata']['url'],
    name=model_asset_details['entity']['name'],
    description=model_asset_details['entity']['description'],
    deployment_type=model_asset_details['entity']['type'],
    scoring_endpoint = scoring_endpoint
) 

In [29]:
training_data_reference = TrainingDataReference(type='cos',
                                              location=COSTrainingDataReferenceLocation(bucket = BUCKET_NAME,
                                                                                        file_name = training_data_file_name),
                                              connection=COSTrainingDataReferenceConnection(
                                                                        resource_instance_id= COS_RESOURCE_CRN,
                                                                        url= COS_ENDPOINT,
                                                                        api_key= COS_API_KEY_ID,
                                                                        iam_url=IAM_URL)
                                               )
os = model_asset_details['entity']['asset_properties']['output_data_schema']
input_s= model_asset_details['entity']['asset_properties']['input_data_schema']
input_schema = SparkStruct(
        type=model_asset_details['entity']['asset_properties']['input_data_schema']['type'],
        fields=model_asset_details['entity']['asset_properties']['input_data_schema']['fields']
    )
output_schema = SparkStruct(
        type=model_asset_details['entity']['asset_properties']['output_data_schema']['type'],
        fields=get_output_schema_fields_for_azure(model_asset_details['entity']['asset_properties']['output_data_schema'], target_field = "Scored Labels", target_role = "decoded-target")
    )
output_schema.to_dict()

{'type': 'struct',
 'fields': [{'name': 'CheckingStatus', 'type': 'string', 'nullable': False},
  {'name': 'LoanDuration', 'type': 'integer', 'nullable': False},
  {'name': 'CreditHistory', 'type': 'string', 'nullable': False},
  {'name': 'LoanPurpose', 'type': 'string', 'nullable': False},
  {'name': 'LoanAmount', 'type': 'integer', 'nullable': False},
  {'name': 'ExistingSavings', 'type': 'string', 'nullable': False},
  {'name': 'EmploymentDuration', 'type': 'string', 'nullable': False},
  {'name': 'InstallmentPercent', 'type': 'integer', 'nullable': False},
  {'name': 'Sex', 'type': 'string', 'nullable': False},
  {'name': 'OthersOnLoan', 'type': 'string', 'nullable': False},
  {'name': 'CurrentResidenceDuration', 'type': 'integer', 'nullable': False},
  {'name': 'OwnsProperty', 'type': 'string', 'nullable': False},
  {'name': 'Age', 'type': 'integer', 'nullable': False},
  {'name': 'InstallmentPlans', 'type': 'string', 'nullable': False},
  {'name': 'Housing', 'type': 'string', 'nu

In [30]:
asset_properties = AssetPropertiesRequest(
        label_column="Risk",
        prediction_field='Scored Labels',
        probability_fields=['Scored Probabilities'],
        training_data_reference=training_data_reference,
        training_data_schema=None,
        input_data_schema=input_schema,
        output_data_schema=output_schema,
        feature_fields=feature_columns,
        categorical_fields=categorical_columns
    )

In [31]:
subscription_details = wos_client.subscriptions.add(
        data_mart_id=data_mart_id,
        service_provider_id=service_provider_id,
        asset=azure_asset,
        deployment=deployment,
        asset_properties=asset_properties,
        background_mode=False
).result
subscription_id = subscription_details.metadata.id
subscription_id




 Waiting for end of adding subscription 37104d6f-4057-4d06-8725-91bf00bd9236 




active

-------------------------------------------
 Successfully finished adding subscription 
-------------------------------------------




'37104d6f-4057-4d06-8725-91bf00bd9236'

<a id="scoring"></a>
# 4. Scoring and payload logging

### Score the credit risk model and measure response time

In [32]:
import time

time.sleep(5)
payload_data_set_id = None
payload_data_set_id = wos_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
if payload_data_set_id is None:
    print("Payload data set not found. Please check subscription status.")
else:
    print("Payload data set id: ", payload_data_set_id)

Payload data set id:  ec79a324-1e3d-42ec-986a-4af9943019c2


In [33]:
import requests
import time
import json

subscription_details=wos_client.subscriptions.get(subscription_id).result.to_dict()
scoring_url = subscription_details['entity']['deployment']['scoring_endpoint']['url']

data = {
            "Inputs": {
                "input1":
                    training_data_pd.sample(2).drop(['Risk'],axis=1).to_dict('records'),
            },
            "GlobalParameters": {
            }
        }

body = str.encode(json.dumps(data))

#token = subscription_details['entity']['deployment']['scoring_endpoint']['credentials']['secret_id']
token = model_asset_details['entity']['scoring_endpoint']['credentials']['token']
headers = subscription_details['entity']['deployment']['scoring_endpoint']['request_headers']
headers['Authorization'] = ('Bearer ' + token)

start_time = time.time()
response = requests.post(url=scoring_url, data=body, headers=headers)
response_time = int(time.time() - start_time)*1000
result = response.json()

print(json.dumps(result, indent=2))

{
  "Results": {
    "output1": [
      {
        "CheckingStatus": "0_to_200",
        "LoanDuration": "49",
        "CreditHistory": "outstanding_credit",
        "LoanPurpose": "repairs",
        "LoanAmount": "4722",
        "ExistingSavings": "greater_1000",
        "EmploymentDuration": "greater_7",
        "InstallmentPercent": "5",
        "Sex": "male",
        "OthersOnLoan": "co-applicant",
        "CurrentResidenceDuration": "3",
        "OwnsProperty": "unknown",
        "Age": "37",
        "InstallmentPlans": "none",
        "Housing": "own",
        "ExistingCreditsCount": "2",
        "Job": "skilled",
        "Dependents": "1",
        "Telephone": "yes",
        "ForeignWorker": "no",
        "Scored Labels": "Risk",
        "Scored Probabilities": "0.999970197677612"
      },
      {
        "CheckingStatus": "no_checking",
        "LoanDuration": "23",
        "CreditHistory": "outstanding_credit",
        "LoanPurpose": "car_used",
        "LoanAmount": "3000",
  

## methods to adjust payload data format for OpenScale 

In [34]:
def convert_user_input_2_openscale(input_data):
    users_records = input_data['Inputs']['input1']
    openscale_fields = list(users_records[0])
    openscale_values = [[rec[k] for k in openscale_fields] for rec in users_records] 

    return {'fields':openscale_fields, 'values':openscale_values}

In [35]:
import numpy as np
def convert_user_output_2_openscale(output_data):
    users_records = output_data['Results']['output1']
    openscale_fields = ["Scored Labels","Scored Probabilities"]
    openscale_values = [[np.double(rec[k]) if k=='Scored Probabilities' else rec[k] for k in openscale_fields ] for rec in users_records] 

    return {'fields':openscale_fields, 'values':openscale_values}

In [36]:
openscale_input=convert_user_input_2_openscale(data)
openscale_output=convert_user_output_2_openscale(result)

### Store the request and response in payload logging table

In [37]:
openscale_output

{'fields': ['Scored Labels', 'Scored Probabilities'],
 'values': [['Risk', 0.999970197677612], ['Risk', 0.999994814395905]]}

In [38]:
import uuid
from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord

print("Performing explicit payload logging.....")
wos_client.data_sets.store_records(data_set_id=payload_data_set_id, request_body=[PayloadRecord(
           scoring_id=str(uuid.uuid4()),
           request=openscale_input,
           response=openscale_output,
           response_time=460
)])
time.sleep(5)
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))

Performing explicit payload logging.....
Number of records in the payload logging table: 2


In [39]:
wos_client.data_sets.show_records(data_set_id=payload_data_set_id)

None,4722,79f8e455-d626-4a29-aa48-89affedfb19c-1,5,greater_1000,2020-11-03T19:26:31.071Z,0.999970197677612,"[2.9802322388e-05, 0.999970197677612]",0_to_200,repairs,0.999970197677612,skilled,none,co-applicant,49,37,outstanding_credit,3,no,own,2,6145fc17a53dc0223d90aaf45b7cd2b7,yes,Risk,male,1,unknown,greater_7
None,3000,79f8e455-d626-4a29-aa48-89affedfb19c-2,3,greater_1000,2020-11-03T19:26:31.071Z,0.999994814395905,"[5.185604095e-06, 0.999994814395905]",no_checking,car_used,0.999994814395905,skilled,none,guarantor,23,61,outstanding_credit,5,no,own,1,6145fc17a53dc0223d90aaf45b7cd2b7,yes,Risk,male,1,unknown,4_to_7


<a id="feedback"></a>
# 5. Feedback logging & quality (accuracy) monitoring

### Enable quality monitoring

The code below waits ten seconds to allow the payload logging table to be set up before it begins enabling monitors. First, it turns on the quality (accuracy) monitor and sets an alert threshold of 70%. OpenScale will show an alert on the dashboard if the model accuracy measurement (area under the curve, in the case of a binary classifier) falls below this threshold.

The second paramater supplied, min_records, specifies the minimum number of feedback records OpenScale needs before it calculates a new measurement. The quality monitor runs hourly, but the accuracy reading in the dashboard will not change until an additional 50 feedback records have been added, via the user interface, the Python client, or the supplied feedback endpoint.

In [40]:
import time

time.sleep(10)
target = Target(
        target_type=TargetTypes.SUBSCRIPTION,
        target_id=subscription_id
)
parameters = {
    "min_feedback_data_size": 10
}
thresholds = [
                {
                    "metric_id": "area_under_roc",
                    "type": "lower_limit",
                    "value": .80
                }
            ]
quality_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result




 Waiting for end of monitor instance creation d3422ba7-85d4-4a1a-a0ae-2092f0988f6d 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




In [41]:
quality_monitor_instance_id = quality_monitor_details.metadata.id
quality_monitor_instance_id

'd3422ba7-85d4-4a1a-a0ae-2092f0988f6d'

### Feedback records logging

Feedback records are used to evaluate your model. The predicted values are compared to real values (feedback records).

In [42]:
!rm additional_feedback_data_v2.json
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/assets/data/credit_risk/additional_feedback_data_v2.json

--2020-11-03 14:28:05--  https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/assets/data/credit_risk/additional_feedback_data_v2.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.124.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.124.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50890 (50K) [text/plain]
Saving to: ‘additional_feedback_data_v2.json’

additional_feedback 100%[===================>]  49.70K  --.-KB/s    in 0.05s   

2020-11-03 14:28:06 (1.04 MB/s) - ‘additional_feedback_data_v2.json’ saved [50890/50890]



### Get feedback logging dataset ID

In [43]:
feedback_dataset_id = None
feedback_dataset = wos_client.data_sets.list(type=DataSetTypes.FEEDBACK, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result
#print(feedback_dataset)
feedback_dataset_id = feedback_dataset.data_sets[0].metadata.id
if feedback_dataset_id is None:
    print("Feedback data set not found. Please check quality monitor status.")

In [44]:
with open('additional_feedback_data_v2.json') as feedback_file:
    additional_feedback_data = json.load(feedback_file)
wos_client.data_sets.store_records(feedback_dataset_id, request_body=additional_feedback_data, background_mode=False)

[{'CheckingStatus': 'less_0',
  'LoanDuration': 10,
  'CreditHistory': 'all_credits_paid_back',
  'LoanPurpose': 'car_new',
  'LoanAmount': 250,
  'ExistingSavings': '500_to_1000',
  'EmploymentDuration': '4_to_7',
  'InstallmentPercent': 3,
  'Sex': 'male',
  'OthersOnLoan': 'none',
  'CurrentResidenceDuration': 2,
  'OwnsProperty': 'real_estate',
  'Age': 23,
  'InstallmentPlans': 'none',
  'Housing': 'rent',
  'ExistingCreditsCount': 1,
  'Job': 'skilled',
  'Dependents': 1,
  'Telephone': 'none',
  'ForeignWorker': 'yes',
  'Risk': 'No Risk'},
 {'CheckingStatus': 'no_checking',
  'LoanDuration': 23,
  'CreditHistory': 'prior_payments_delayed',
  'LoanPurpose': 'appliances',
  'LoanAmount': 6964,
  'ExistingSavings': '100_to_500',
  'EmploymentDuration': '4_to_7',
  'InstallmentPercent': 4,
  'Sex': 'female',
  'OthersOnLoan': 'none',
  'CurrentResidenceDuration': 3,
  'OwnsProperty': 'car_other',
  'Age': 39,
  'InstallmentPlans': 'none',
  'Housing': 'own',
  'ExistingCreditsCount

In [51]:
# manually records scoring and store as feedback data

# records = [
#     ["no_checking","28","outstanding_credit","appliances","5990","500_to_1000","greater_7","5","male","co-applicant","3","car_other","55","none","free","2","skilled","2","yes","yes","Risk"],
#     ["greater_200","22","all_credits_paid_back","car_used","3376","less_100","less_1","3","female","none","2","car_other","32","none","own","1","skilled","1","none","yes","No Risk"],
#     ["no_checking","39","credits_paid_to_date","vacation","6434","unknown","greater_7","5","male","none","4","car_other","39","none","own","2","skilled","2","yes","yes","Risk"],
#     ["0_to_200","20","credits_paid_to_date","furniture","2442","less_100","unemployed","3","female","none","1","real_estate","42","none","own","1","skilled","1","none","yes","No Risk"],
#     ["greater_200","4","all_credits_paid_back","education","4206","less_100","unemployed","1","female","none","3","savings_insurance","27","none","own","1","management_self-employed","1","none","yes","No Risk"],
#     ["greater_200","23","credits_paid_to_date","car_used","2963","greater_1000","greater_7","4","male","none","4","car_other","46","none","own","2","skilled","1","none","yes","Risk"],
#     ["no_checking","31","prior_payments_delayed","vacation","2673","500_to_1000","1_to_4","3","male","none","2","real_estate","35","stores","rent","1","skilled","2","none","yes","Risk"],
#     ["no_checking","37","prior_payments_delayed","other","6971","500_to_1000","1_to_4","3","male","none","3","savings_insurance","54","none","own","2","skilled","1","yes","yes","Risk"],
#     ["0_to_200","39","prior_payments_delayed","appliances","5685","100_to_500","1_to_4","4","female","none","2","unknown","37","none","own","2","skilled","1","yes","yes","Risk"],
#     ["no_checking","38","prior_payments_delayed","appliances","4990","500_to_1000","greater_7","4","male","none","4","car_other","50","bank","own","2","unemployed","2","yes","yes","Risk"]]

# fields = feature_columns.copy()
# fields.append('Risk')

# payload_scoring =  [{"fields": fields, "values": records}]
#wos_client.data_sets.store_records(feedback_dataset_id, request_body=payload_scoring, background_mode=False)

In [46]:
wos_client.data_sets.get_records_count(data_set_id=feedback_dataset_id)

98

In [47]:
run_details = wos_client.monitor_instances.run(monitor_instance_id=quality_monitor_instance_id, background_mode=False).result




 Waiting for end of monitoring run 63b164d0-5a9b-410b-bfd4-7144f21861ce 




running
finished

---------------------------
 Successfully finished run 
---------------------------




In [48]:
time.sleep(5)
wos_client.monitor_instances.show_metrics(monitor_instance_id=quality_monitor_instance_id)

# 6. Fairness, Drift monitoring and explanations <a id="monitors"></a>

### Fairness monitor configuration

In [49]:
wos_client.monitor_instances.show()

5a0b9076-fcf6-49e8-a824-9e3a6b4c2a56,active,37104d6f-4057-4d06-8725-91bf00bd9236,subscription,quality,2020-11-03 19:27:34.491000+00:00,d3422ba7-85d4-4a1a-a0ae-2092f0988f6d
5a0b9076-fcf6-49e8-a824-9e3a6b4c2a56,active,a337d621-5de4-4c63-8d83-fd02d6db14af,subscription,drift,2020-11-03 01:07:00.118000+00:00,f085f563-c798-4243-9277-583bfb216e09
5a0b9076-fcf6-49e8-a824-9e3a6b4c2a56,active,a337d621-5de4-4c63-8d83-fd02d6db14af,subscription,fairness,2020-11-03 01:06:45.377000+00:00,f77cb0ef-399b-457d-b461-bf1983ca3407
5a0b9076-fcf6-49e8-a824-9e3a6b4c2a56,active,887596d0-87f9-4eaf-9ee9-30b87d5d03ba,subscription,fairness,2020-11-02 23:01:57.117000+00:00,74686430-eee1-44f3-9eee-02316109e9d1
5a0b9076-fcf6-49e8-a824-9e3a6b4c2a56,active,72a076d6-fb87-4f52-96b8-891cd405c6e1,subscription,fairness,2020-10-21 20:57:06.622000+00:00,bdef083f-15f2-4c94-b724-3e278d4e0269
5a0b9076-fcf6-49e8-a824-9e3a6b4c2a56,active,887596d0-87f9-4eaf-9ee9-30b87d5d03ba,subscription,drift,2020-11-02 23:02:05.578000+00:00,4eda9182-c990-49b9-a46b-de057b7d29ad
5a0b9076-fcf6-49e8-a824-9e3a6b4c2a56,active,887596d0-87f9-4eaf-9ee9-30b87d5d03ba,subscription,quality,2020-11-02 22:57:32.863000+00:00,78bec9c2-b177-4606-96b0-3d25cf8a629b
5a0b9076-fcf6-49e8-a824-9e3a6b4c2a56,active,72a076d6-fb87-4f52-96b8-891cd405c6e1,subscription,mrm,2020-10-21 21:04:44.385000+00:00,b67f7336-b8d7-43c2-8318-063cdc01e122
5a0b9076-fcf6-49e8-a824-9e3a6b4c2a56,active,4652fbc2-fdee-47a0-b58e-a440ab5d22c8,instance,performance,2020-09-03 21:51:45.075000+00:00,b25fd34b-5b85-44f0-a950-30c816ecbd5a
5a0b9076-fcf6-49e8-a824-9e3a6b4c2a56,active,020f8ef9-fdb1-4d4f-89e9-9dad2b30b363,instance,performance,2020-09-03 18:26:04.634000+00:00,da90f8a6-0ead-4d6e-83b6-3dc6f0c90551


Note: First 10 records were displayed.


In [50]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id

)
parameters = {
    "features": [
        {"feature": "Sex",
         "majority": ['male'],
         "minority": ['female'],
         "threshold": 0.95
         },
        {"feature": "Age",
         "majority": [[26, 75]],
         "minority": [[18, 25]],
         "threshold": 0.95
         }
    ],
    "favourable_class": ["No Risk"],
    "unfavourable_class": ["Risk"],
    "min_records": 40
}

fairness_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.FAIRNESS.ID,
    target=target,
    parameters=parameters).result
fairness_monitor_instance_id =fairness_monitor_details.metadata.id
fairness_monitor_instance_id




 Waiting for end of monitor instance creation 81c4a69f-68bd-4ccb-9427-cdfb0d32804e 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




'81c4a69f-68bd-4ccb-9427-cdfb0d32804e'

### Payload logging and Run fairness monitor on demand

In [52]:
import requests
import time
import json

scoring_url = subscription_details['entity']['deployment']['scoring_endpoint']['url']

payload_data = {
            "Inputs": {
                "input1":
                    training_data_pd.sample(40).drop(['Risk'],axis=1).to_dict('records'),
            },
            "GlobalParameters": {
            }
        }

pl_body = str.encode(json.dumps(payload_data))

#token = subscription_details['entity']['deployment']['scoring_endpoint']['credentials']['secret_id']
token = model_asset_details['entity']['scoring_endpoint']['credentials']['token']
headers = subscription_details['entity']['deployment']['scoring_endpoint']['request_headers']
headers['Authorization'] = ('Bearer ' + token)

start_time = time.time()
response = requests.post(url=scoring_url, data=pl_body, headers=headers)
response_time = int(time.time() - start_time)*1000
result = response.json()

print(json.dumps(result, indent=2))

{
  "Results": {
    "output1": [
      {
        "CheckingStatus": "0_to_200",
        "LoanDuration": "19",
        "CreditHistory": "prior_payments_delayed",
        "LoanPurpose": "car_new",
        "LoanAmount": "4434",
        "ExistingSavings": "100_to_500",
        "EmploymentDuration": "4_to_7",
        "InstallmentPercent": "3",
        "Sex": "female",
        "OthersOnLoan": "guarantor",
        "CurrentResidenceDuration": "2",
        "OwnsProperty": "car_other",
        "Age": "43",
        "InstallmentPlans": "stores",
        "Housing": "free",
        "ExistingCreditsCount": "1",
        "Job": "unskilled",
        "Dependents": "1",
        "Telephone": "yes",
        "ForeignWorker": "yes",
        "Scored Labels": "Risk",
        "Scored Probabilities": "0.782753229141235"
      },
      {
        "CheckingStatus": "less_0",
        "LoanDuration": "20",
        "CreditHistory": "prior_payments_delayed",
        "LoanPurpose": "retraining",
        "LoanAmount": "23

### Store the request and response in payload logging table

In [53]:
openscale_paylod_input=convert_user_input_2_openscale(payload_data)
openscale_payload_output=convert_user_output_2_openscale(result)

In [54]:
import uuid
from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord

print("Performing explicit payload logging.....")
wos_client.data_sets.store_records(data_set_id=payload_data_set_id, request_body=[PayloadRecord(
           scoring_id=str(uuid.uuid4()),
           request=openscale_paylod_input,
           response=openscale_payload_output,
           response_time=460
)])
time.sleep(5)
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))

Performing explicit payload logging.....
Number of records in the payload logging table: 42


In [55]:
wos_client.data_sets.show_records(payload_data_set_id)

None,4434,4a97742b-879e-4aeb-ba6d-988cd47b69d7-1,3,100_to_500,2020-11-03T19:32:53.577Z,0.782753229141235,"[0.217246770858765, 0.782753229141235]",0_to_200,car_new,0.782753229141235,unskilled,stores,guarantor,19,43,prior_payments_delayed,2,yes,free,1,6145fc17a53dc0223d90aaf45b7cd2b7,yes,Risk,female,1,car_other,4_to_7
None,250,4a97742b-879e-4aeb-ba6d-988cd47b69d7-10,2,less_100,2020-11-03T19:32:53.577Z,0.9999996909342599,"[0.9999996909342599, 3.09065740111691e-07]",less_0,car_used,3.09065740111691e-07,unskilled,bank,none,10,21,all_credits_paid_back,1,yes,rent,1,6145fc17a53dc0223d90aaf45b7cd2b7,none,No Risk,female,1,real_estate,less_1
None,250,4a97742b-879e-4aeb-ba6d-988cd47b69d7-11,2,less_100,2020-11-03T19:32:53.577Z,0.999981334523909,"[0.999981334523909, 1.86654760909732e-05]",0_to_200,furniture,1.86654760909732e-05,skilled,none,none,16,19,credits_paid_to_date,1,yes,own,1,6145fc17a53dc0223d90aaf45b7cd2b7,none,No Risk,female,1,real_estate,less_1
None,252,4a97742b-879e-4aeb-ba6d-988cd47b69d7-12,2,less_100,2020-11-03T19:32:53.577Z,0.9999367315685959,"[0.9999367315685959, 6.32684314041398e-05]",0_to_200,education,6.32684314041398e-05,skilled,stores,none,4,34,prior_payments_delayed,1,yes,rent,1,6145fc17a53dc0223d90aaf45b7cd2b7,none,No Risk,female,1,savings_insurance,1_to_4
None,7200,4a97742b-879e-4aeb-ba6d-988cd47b69d7-13,3,100_to_500,2020-11-03T19:32:53.577Z,0.952577292919159,"[0.047422707080841, 0.952577292919159]",no_checking,vacation,0.952577292919159,management_self-employed,none,none,13,35,credits_paid_to_date,4,yes,own,2,6145fc17a53dc0223d90aaf45b7cd2b7,yes,Risk,male,2,car_other,1_to_4
None,2581,4a97742b-879e-4aeb-ba6d-988cd47b69d7-14,3,100_to_500,2020-11-03T19:32:53.577Z,0.999366941046901,"[0.999366941046901, 0.000633058953098953]",0_to_200,car_used,0.000633058953098953,skilled,none,none,23,41,outstanding_credit,2,yes,own,1,6145fc17a53dc0223d90aaf45b7cd2b7,yes,No Risk,male,2,savings_insurance,less_1
None,3942,4a97742b-879e-4aeb-ba6d-988cd47b69d7-15,4,100_to_500,2020-11-03T19:32:53.577Z,0.633385896682739,"[0.366614103317261, 0.633385896682739]",greater_200,car_used,0.633385896682739,skilled,none,none,28,46,prior_payments_delayed,3,no,free,2,6145fc17a53dc0223d90aaf45b7cd2b7,none,Risk,male,1,savings_insurance,4_to_7
None,5790,4a97742b-879e-4aeb-ba6d-988cd47b69d7-16,5,500_to_1000,2020-11-03T19:32:53.577Z,0.9526074081659317,"[0.9526074081659317, 0.0473925918340683]",greater_200,other,0.0473925918340683,unskilled,stores,none,34,44,outstanding_credit,4,yes,own,2,6145fc17a53dc0223d90aaf45b7cd2b7,yes,No Risk,male,1,car_other,greater_7
None,6610,4a97742b-879e-4aeb-ba6d-988cd47b69d7-17,4,100_to_500,2020-11-03T19:32:53.577Z,0.9355000108480453,"[0.9355000108480453, 0.0644999891519547]",no_checking,appliances,0.0644999891519547,skilled,none,none,31,48,prior_payments_delayed,3,yes,own,2,6145fc17a53dc0223d90aaf45b7cd2b7,none,No Risk,female,1,savings_insurance,1_to_4
None,2673,4a97742b-879e-4aeb-ba6d-988cd47b69d7-18,3,500_to_1000,2020-11-03T19:32:53.577Z,0.540772378444672,"[0.540772378444672, 0.459227621555328]",no_checking,vacation,0.459227621555328,skilled,stores,none,31,35,prior_payments_delayed,2,yes,rent,1,6145fc17a53dc0223d90aaf45b7cd2b7,none,No Risk,male,2,real_estate,1_to_4


In [56]:
run_details = wos_client.monitor_instances.run(monitor_instance_id=fairness_monitor_instance_id, background_mode=False)




 Waiting for end of monitoring run 37ff37d5-f4c5-46a0-884c-0ab5ed8aa5b3 




running.
finished

---------------------------
 Successfully finished run 
---------------------------




In [57]:
time.sleep(10)

wos_client.monitor_instances.show_metrics(monitor_instance_id=fairness_monitor_instance_id)

### Explainability configuration

In [58]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)
parameters = {
    "enabled": True
}
explainability_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target,
    parameters=parameters
).result




 Waiting for end of monitor instance creation 5b1cddc6-de58-4611-9f00-26c75f7b50e8 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




In [59]:
explainability_monitor_id = explainability_details.metadata.id
explainability_monitor_id

'5b1cddc6-de58-4611-9f00-26c75f7b50e8'

### Run explanation

In [60]:
wos_client.data_sets.show_records(data_set_id=payload_data_set_id)

None,"[0.217246770858765, 0.782753229141235]",4434,4a97742b-879e-4aeb-ba6d-988cd47b69d7-1,3,100_to_500,2020-11-03T19:32:53.577Z,0.782753229141235,"[0.217246770858765, 0.782753229141235]",0_to_200,car_new,0.782753229141235,unskilled,stores,guarantor,19,43,prior_payments_delayed,2,yes,free,1,6145fc17a53dc0223d90aaf45b7cd2b7,yes,Risk,female,1,car_other,4_to_7,Risk
None,"[0.9999996909342599, 3.09065740111691e-07]",250,4a97742b-879e-4aeb-ba6d-988cd47b69d7-10,2,less_100,2020-11-03T19:32:53.577Z,0.9999996909342599,"[0.9999996909342599, 3.09065740111691e-07]",less_0,car_used,3.09065740111691e-07,unskilled,bank,none,10,21,all_credits_paid_back,1,yes,rent,1,6145fc17a53dc0223d90aaf45b7cd2b7,none,No Risk,female,1,real_estate,less_1,No Risk
None,"[0.999981334523909, 1.86654760909732e-05]",250,4a97742b-879e-4aeb-ba6d-988cd47b69d7-11,2,less_100,2020-11-03T19:32:53.577Z,0.999981334523909,"[0.999981334523909, 1.86654760909732e-05]",0_to_200,furniture,1.86654760909732e-05,skilled,none,none,16,19,credits_paid_to_date,1,yes,own,1,6145fc17a53dc0223d90aaf45b7cd2b7,none,No Risk,female,1,real_estate,less_1,No Risk
None,"[0.9999367315685959, 6.32684314041398e-05]",252,4a97742b-879e-4aeb-ba6d-988cd47b69d7-12,2,less_100,2020-11-03T19:32:53.577Z,0.9999367315685959,"[0.9999367315685959, 6.32684314041398e-05]",0_to_200,education,6.32684314041398e-05,skilled,stores,none,4,34,prior_payments_delayed,1,yes,rent,1,6145fc17a53dc0223d90aaf45b7cd2b7,none,No Risk,female,1,savings_insurance,1_to_4,No Risk
None,"[0.047422707080841, 0.952577292919159]",7200,4a97742b-879e-4aeb-ba6d-988cd47b69d7-13,3,100_to_500,2020-11-03T19:32:53.577Z,0.952577292919159,"[0.047422707080841, 0.952577292919159]",no_checking,vacation,0.952577292919159,management_self-employed,none,none,13,35,credits_paid_to_date,4,yes,own,2,6145fc17a53dc0223d90aaf45b7cd2b7,yes,Risk,male,2,car_other,1_to_4,Risk
None,"[0.999366941046901, 0.000633058953098953]",2581,4a97742b-879e-4aeb-ba6d-988cd47b69d7-14,3,100_to_500,2020-11-03T19:32:53.577Z,0.999366941046901,"[0.999366941046901, 0.000633058953098953]",0_to_200,car_used,0.000633058953098953,skilled,none,none,23,41,outstanding_credit,2,yes,own,1,6145fc17a53dc0223d90aaf45b7cd2b7,yes,No Risk,male,2,savings_insurance,less_1,No Risk
None,"[0.366614103317261, 0.633385896682739]",3942,4a97742b-879e-4aeb-ba6d-988cd47b69d7-15,4,100_to_500,2020-11-03T19:32:53.577Z,0.633385896682739,"[0.366614103317261, 0.633385896682739]",greater_200,car_used,0.633385896682739,skilled,none,none,28,46,prior_payments_delayed,3,no,free,2,6145fc17a53dc0223d90aaf45b7cd2b7,none,Risk,male,1,savings_insurance,4_to_7,Risk
None,"[0.9526074081659317, 0.0473925918340683]",5790,4a97742b-879e-4aeb-ba6d-988cd47b69d7-16,5,500_to_1000,2020-11-03T19:32:53.577Z,0.9526074081659317,"[0.9526074081659317, 0.0473925918340683]",greater_200,other,0.0473925918340683,unskilled,stores,none,34,44,outstanding_credit,4,yes,own,2,6145fc17a53dc0223d90aaf45b7cd2b7,yes,No Risk,male,1,car_other,greater_7,No Risk
None,"[0.9355000108480453, 0.0644999891519547]",6610,4a97742b-879e-4aeb-ba6d-988cd47b69d7-17,4,100_to_500,2020-11-03T19:32:53.577Z,0.9355000108480453,"[0.9355000108480453, 0.0644999891519547]",no_checking,appliances,0.0644999891519547,skilled,none,none,31,48,prior_payments_delayed,3,yes,own,2,6145fc17a53dc0223d90aaf45b7cd2b7,none,No Risk,female,1,savings_insurance,1_to_4,No Risk
None,"[0.540772378444672, 0.459227621555328]",2673,4a97742b-879e-4aeb-ba6d-988cd47b69d7-18,3,500_to_1000,2020-11-03T19:32:53.577Z,0.540772378444672,"[0.540772378444672, 0.459227621555328]",no_checking,vacation,0.459227621555328,skilled,stores,none,31,35,prior_payments_delayed,2,yes,rent,1,6145fc17a53dc0223d90aaf45b7cd2b7,none,No Risk,male,2,real_estate,1_to_4,No Risk


In [61]:
pl_records_resp = wos_client.data_sets.get_list_of_records(data_set_id=payload_data_set_id, limit=1, offset=0).result
scoring_ids = [pl_records_resp["records"][0]["entity"]["values"]["scoring_id"]]
print("Running explanations on scoring IDs: {}".format(scoring_ids))
explanation_types = ["lime", "contrastive"]
result = wos_client.monitor_instances.explanation_tasks(scoring_ids=scoring_ids, explanation_types=explanation_types).result
print(result)
explanation_task_id=result.to_dict()['metadata']['explanation_task_ids'][0]
explanation_task_id

Running explanations on scoring IDs: ['4a97742b-879e-4aeb-ba6d-988cd47b69d7-1']
{
  "metadata": {
    "explanation_task_ids": [
      "b750794c-02c9-4972-993f-6c7c0868e4ea"
    ],
    "created_by": "IBMid-310002F0G1",
    "created_at": "2020-11-03T19:34:12.035101Z"
  }
}


'b750794c-02c9-4972-993f-6c7c0868e4ea'

In [64]:
wos_client.monitor_instances.get_explanation_tasks(explanation_task_id=explanation_task_id).result.to_dict()

{'metadata': {'explanation_task_id': 'b750794c-02c9-4972-993f-6c7c0868e4ea',
  'created_by': 'IBMid-310002F0G1',
  'created_at': '2020-11-03T19:34:12.035101Z',
  'updated_at': '2020-11-03T19:34:27.410900Z'},
 'entity': {'status': {'state': 'in_progress'},
  'asset': {'id': '4fff12e2b55fe1038b2e04d98be759c5',
   'name': 'FastpathGermanCreditRisk.2019.2.1.17.17.21.412',
   'input_data_type': 'structured',
   'problem_type': 'binary',
   'deployment': {'id': '6145fc17a53dc0223d90aaf45b7cd2b7',
    'name': 'FastpathGermanCreditRisk.2019.2.1.17.17.21.412'}},
  'input_features': [{'name': 'CheckingStatus',
    'value': '0_to_200',
    'feature_type': 'categorical'},
   {'name': 'LoanDuration', 'value': '19', 'feature_type': 'numerical'},
   {'name': 'CreditHistory',
    'value': 'prior_payments_delayed',
    'feature_type': 'categorical'},
   {'name': 'LoanPurpose', 'value': 'car_new', 'feature_type': 'categorical'},
   {'name': 'LoanAmount', 'value': '4434', 'feature_type': 'numerical'},
  

### Drift monitor configuration

#### Drift requires a trained model to be uploaded manually for Azure. You can train, create and download a drift detection model using template given ( check for Drift detection model generation) [here](https://github.com/IBM-Watson/aios-data-distribution/blob/master/training_statistics_notebook.ipynb)

In [67]:
!rm -rf creditrisk_azure_drift_detection_model.tar.gz
!wget -O creditrisk_azure_drift_detection_model.tar.gz https://github.com/IBM/watson-openscale-samples/blob/main/assets/models/credit_risk/azure_creditrisk_drift_detection_model.tar.gz?raw=true 

--2020-11-03 15:16:50--  https://github.com/IBM/watson-openscale-samples/blob/main/assets/models/credit_risk/azure_creditrisk_drift_detection_model.tar.gz?raw=true
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/IBM/watson-openscale-samples/raw/main/assets/models/credit_risk/azure_creditrisk_drift_detection_model.tar.gz [following]
--2020-11-03 15:16:51--  https://github.com/IBM/watson-openscale-samples/raw/main/assets/models/credit_risk/azure_creditrisk_drift_detection_model.tar.gz
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/assets/models/credit_risk/azure_creditrisk_drift_detection_model.tar.gz [following]
--2020-11-03 15:16:51--  https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/assets/mo

In [68]:
wos_client.monitor_instances.upload_drift_model(
        model_path='creditrisk_azure_drift_detection_model.tar.gz',
        data_mart_id=data_mart_id,
        subscription_id=subscription_id
     )

In [69]:
monitor_instances = wos_client.monitor_instances.list().result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == "drift" and monitor_instance.entity.target.target_id == subscription_id:
        wos_client.monitor_instances.delete(monitor_instance.metadata.id)
        print('Deleted existing drift monitor instance with id: ', monitor_instance.metadata.id)

In [70]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id

)
parameters = {
    "min_samples": 40,
    "drift_threshold": 0.1,
    "train_drift_model": False,
    "enable_model_drift": True,
    "enable_data_drift": True
}

drift_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT.ID,
    target=target,
    parameters=parameters
).result

drift_monitor_instance_id = drift_monitor_details.metadata.id
drift_monitor_instance_id




 Waiting for end of monitor instance creation 8d460af5-d2fd-4116-b231-2d754efe469f 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




'8d460af5-d2fd-4116-b231-2d754efe469f'

### Run Drift monitor on demand

In [71]:
drift_run_details = wos_client.monitor_instances.run(monitor_instance_id=drift_monitor_instance_id, background_mode=False)




 Waiting for end of monitoring run 781abf9a-3260-4228-af7b-ab8d65d83b85 




finished

---------------------------
 Successfully finished run 
---------------------------




In [72]:
time.sleep(5)
wos_client.monitor_instances.show_metrics(monitor_instance_id=drift_monitor_instance_id)

## Congratulations!

You have finished the tutorial for IBM Watson OpenScale and Azure Machine Learning Studio. You can now view the [OpenScale Dashboard](https://aiopenscale.cloud.ibm.com/). Click on the tile for the German Credit model to see fairness, accuracy, and performance monitors. Click on the timeseries graph to get detailed information on transactions during a specific time window.



---